In [84]:
STATION="Tošin Bunar" #station name

TIMETABLE_DATE = "30.10.2025" #for date
TIME_FROM = "0000" #000=all
TIME_TO = "999" #999=all
TIMETABLE_LANG="sr" #timetable language

In [31]:
import requests
import urllib.parse
from bs4 import BeautifulSoup

In [5]:
API_BASE_URL="https://w3.srbvoz.rs/redvoznje//api"
WEB_BASE_URL="https://w3.srbvoz.rs/redvoznje"

In [85]:
res = requests.get(f"{API_BASE_URL}/stanica/?term=")

if res.status_code != 200:
    raise Exception(f"Api error occured: {res.status_code}, {res.text}")

stations_ = res.json()

stations = list(map(lambda s: { \
    "name": s["naziv"], 
    "id": s["sifra"], 
    "safe name": s["naziv"].upper().replace(" ", "_").replace("Č", "C").replace("Ć", "C").replace("Š", "S").replace("Đ", "DJ").replace("Ž", "Z") 
}, stations_))

In [82]:
emum = ""

for i in range(0, len(stations)):
    emum += f"{stations[i]["safe name"]} = {stations[i]}\r\n"

emum

"KEMENDIN_ST = {'name': 'Kemendin st', 'id': '01016', 'safe name': 'KEMENDIN_ST'}\r\nALTINA_ST = {'name': 'Altina st', 'id': '01017', 'safe name': 'ALTINA_ST'}\r\nDOLJEVAC = {'name': 'DOLJEVAC', 'id': '11001', 'safe name': 'DOLJEVAC'}\r\nKOCANE = {'name': 'KOCANE', 'id': '11002', 'safe name': 'KOCANE'}\r\nPUKOVAC = {'name': 'PUKOVAC', 'id': '11003', 'safe name': 'PUKOVAC'}\r\nBRESTOVAC = {'name': 'BRESTOVAC', 'id': '11004', 'safe name': 'BRESTOVAC'}\r\nLIPOVICA = {'name': 'LIPOVICA', 'id': '11005', 'safe name': 'LIPOVICA'}\r\nPECENJEVCE = {'name': 'PEČENJEVCE', 'id': '11006', 'safe name': 'PECENJEVCE'}\r\nZIVKOVO = {'name': 'ŽIVKOVO', 'id': '11007', 'safe name': 'ZIVKOVO'}\r\nLESKOVAC = {'name': 'LESKOVAC', 'id': '11050', 'safe name': 'LESKOVAC'}\r\nSAJINOVAC = {'name': 'ŠAJINOVAC', 'id': '11101', 'safe name': 'SAJINOVAC'}\r\nJASENICA = {'name': 'JASENICA', 'id': '11102', 'safe name': 'JASENICA'}\r\nRECICA = {'name': 'REČICA', 'id': '11104', 'safe name': 'RECICA'}\r\nPODINA = {'name': 

In [88]:
#get stations using api, find set station id and create url

station_ = STATION.upper().strip()

try:
    station_id = list(filter(lambda s: s["name"] == station_, stations))[0]["id"]
except:
    raise Exception("Station not found")

station_url = f"{WEB_BASE_URL}//stanicni/{urllib.parse.quote(station_)}/{station_id}"

station_url

'https://w3.srbvoz.rs/redvoznje//stanicni/TO%C5%A0IN%20BUNAR/16012'

In [89]:
# get timetable

timetable = {}

for direction in ["inbound", "outbound"]:
    timetable[direction] = []

    url = f"{station_url}/{TIMETABLE_DATE}/{TIME_FROM}/{"dolazak" if direction == "inbound" else "polazak"}/{TIME_TO}/{TIMETABLE_LANG}"

    res = requests.get(url)

    if res.status_code != 200:
        raise Exception(f"Could not get timetable: {res.status_code}, {res.text}, url: {url}")

    #parse html
    html = BeautifulSoup(res.text, 'lxml')
    
    #get table from html
    table_rows = html.select("#rezultati > table > tr.tsmall")

    #get headers
    table_headers = list(map(lambda h: h.get_text(strip=True), table_rows[0].find_all("th")))[:-1]
    table_headers.append("Details")

    #for each row (skipping first row with headers), create dict and save in list

    for row in table_rows[1:]:

        #get data from row
        data = list(map(lambda d: d.get_text(strip=True), row.select("td")))
    
        timetable_row = {}

        #go throught the headers
        for i in range(0, len(table_headers)):
            timetable_row[table_headers[i]] = data[i] if i < len(data) else '' #handle if no data for header

        timetable[direction].append(timetable_row)
    # html.find()

timetable

{'inbound': [{'Broj voza': '2421',
   'Vreme polaska': '23:45',
   'Stanica': 'NOVI SAD',
   'Vreme dolaska': '00:36',
   'Rang': '',
   'Ponuda': '',
   'Kasni': '',
   'Napomena': 'Saobraca od stanice Petrovaradin',
   'Details': 'Detaljnije'},
  {'Broj voza': '7901',
   'Vreme polaska': '03:44',
   'Stanica': 'ZEMUN',
   'Vreme dolaska': '03:47',
   'Rang': '',
   'Ponuda': '',
   'Kasni': '',
   'Napomena': '',
   'Details': 'Detaljnije'},
  {'Broj voza': '7101',
   'Vreme polaska': '04:10',
   'Stanica': 'ZEMUN',
   'Vreme dolaska': '04:13',
   'Rang': '',
   'Ponuda': '',
   'Kasni': '',
   'Napomena': '',
   'Details': 'Detaljnije'},
  {'Broj voza': '6001',
   'Vreme polaska': '04:15',
   'Stanica': 'ZEMUN',
   'Vreme dolaska': '04:18',
   'Rang': '',
   'Ponuda': '',
   'Kasni': '',
   'Napomena': 'Ne saob. ned i 1, 2 i 7 jan,2025, 15, 16 i 17 feb, 18, 20 i 21 apr, 1 i 2 maj , 11 nov 2025',
   'Details': 'Detaljnije'},
  {'Broj voza': '2400',
   'Vreme polaska': '04:30',
   'St